# Introduction

This notebook retrieves and analyzes **Russian court cases** (criminal and administrative) from the **sudrf.ru** court information system. It uses the open-source `sudrfparser` project to:

* automatically open court websites,
* navigate their “Sudebnoe delo” module,
* bypass captchas (optionally via 2Captcha),
* download case metadata and full texts,
* save results as structured JSON files.

## Install a ChromeDriver Version That Matches Your Chrome Browser

Before running the parser, you must install a ChromeDriver version that matches the version of Google Chrome installed on your computer.  
Chrome and ChromeDriver must be **exactly the same major version** (e.g., Chrome 142 → ChromeDriver 142).  
If they don’t match, Selenium will fail with an error like:

> This version of ChromeDriver only supports Chrome version XXX.  
> Current browser version is YYY.

Installing dependencies matching versions expected by the parser:

In [ ]:
!pip install --upgrade beautifulsoup4 requests 2captcha-python webdriver_manager "selenium<4.0.0" "urllib3<2.0.0"

Clone the parser repo if you have not before

In [7]:
# !git clone https://github.com/dataout-org/sudrfparser.git

In [8]:
import sys
from pathlib import Path

repo_root = Path.cwd() / "sudrfparser"
assert repo_root.exists(), f"Repo not found at {repo_root}"

sys.path.append(str(repo_root.parent))  # parent folder so 'sudrfparser' package is importable

import sudrfparser.sudrfparser as sp

import inspect
print("Loaded from:", sp.__file__)
print("get_* functions:", [name for name, obj in inspect.getmembers(sp) if inspect.isfunction(obj) and name.startswith("get_")])

Loaded from: /Users/ana/Projects/soas/russian courts/sudrfparser/sudrfparser.py
get_* functions: ['get_adm_cases', 'get_cases', 'get_courts_list']


Here you can specify the website and region and some other filters, as well as TwoCaptcha details if you are using it

In [15]:
import os
from pathlib import Path

# 1) Court info
website = "http://krn.spb.sudrf.ru"   
region  = "78"                             # region code for that court

# 2) Administrative article(s)
# Format in this library: "<article>_<part>"
# Example: КоАП РФ 6.21 ч.1 ⇒ "6.21_1"
adm_article_number = "6.21"   # ← change to what you need
adm_article_part   = "1"      # ← article part
adm_articles = [f"{adm_article_number}_{adm_article_part}"]

# 3) Date range (inclusive) in DD.MM.YYYY
start_date = "01.01.2021"
end_date   = "31.12.2025"

# 4) Where to save result JSONs
path_to_save = Path.cwd() / "sudrf_results"
path_to_save.mkdir(exist_ok=True)
print("Results will be saved into:", path_to_save)

# 5) Optional: TwoCaptcha config (for automatic captcha solving)
# If you don't want auto-captcha, leave this as {} and the script will ask you to type captchas manually
captcha_config = {
    # Uncomment and fill in if you have a key:
    # "config": {
    #     "apiKey": "YOUR_2CAPTCHA_API_KEY_HERE",
    # },
    # # typical options for numeric 5-digit captchas
    # "numeric": 1,
    # "minLen": 5,
    # "maxLen": 5,
    # "phrase": 0,
}

Results will be saved into: /Users/ana/Projects/soas/russian courts/sudrf_results


Gathering administrative cases for the parameters provided. Adjust the path to your driver here if it is different.

In [16]:
results = sp.get_adm_cases(
    website=website,
    region=region,
    adm_articles=adm_articles,
    start_date=start_date,
    end_date=end_date,
    path_to_driver="/opt/homebrew/bin/chromedriver",
    court_code="",          # only needed for form2 sites; leave "" for form1
    srv_num=["1"],          # servers; ["1"] is a good starting point
    path_to_save=str(path_to_save) + os.sep,
    captcha_config=captcha_config,
)

results

Enter captcha:  38484


{'http://krn.spb.sudrf.ru': {'n_cases_by_server': {'1': 0}}}

In [ ]:
import json
import glob

# Get all JSON files for this region
files = sorted(glob.glob(str(path_to_save / f"{region}_*.json")))
print("Result files:", files)

if not files:
    print("No result files found. Check logs / output above.")
else:
    latest = files[-1]
    print("Using file:", latest)

    with open(latest, "r", encoding="utf-8") as f:
        data = json.load(f)

    website_key = list(data.keys())[0]
    site_data = data[website_key]
    cases = site_data.get("cases", [])
    logs  = site_data.get("logs", {})

    print(f"\nTotal cases in this file: {len(cases)}")
    print("Logs:", logs)

    # Show a summary of the first 5 cases
    for i, c in enumerate(cases[:5], start=1):
        md = c.get("metadata", {})
        accused = md.get("accused", [])

        print("\n" + "="*60)
        print(f"Case {i}")
        print("Internal ID:", md.get("id_text", ""))
        print("Decision result:", md.get("decision_result", ""))
        print("Date received:", md.get("adm_date", ""))
        print("Judge:", md.get("judge", ""))

        if accused:
            print("Accused:")
            for a in accused:
                print("  -", a["name"], "| articles:", "; ".join(a["article"]))
        else:
            print("Accused: [none parsed]")

        text = c.get("case_text", "")
        print("Text length:", len(text))
        print("Text excerpt:\n", text[:800])